In [1]:
import numpy as np
import pandas as pd

In [2]:
OUTFILE = f'../data/exam_2_data.xlsx'

***

In [3]:
DATE = '2025-01-14'
DATAPATH_FX = f'../data/fx_rf_data_{DATE}.xlsx'
YRCUT = '2020'

In [4]:
SHEET = 'exchange rates'
fxraw = pd.read_excel(DATAPATH_FX, sheet_name=SHEET).set_index('date')

In [5]:
SHEET = 'risk-free rates'
rfraw = pd.read_excel(DATAPATH_FX,sheet_name=SHEET).set_index('date')

In [6]:
shared_indexes = rfraw.index.intersection(fxraw.index)
# Split the merged DataFrame back into the original two DataFrames with shared indexes
rf = rfraw.loc[shared_indexes,:]
fx = fxraw.loc[shared_indexes,:]

rf = rf.loc[YRCUT:]
fx = fx.loc[YRCUT:]

DAYS = fx.resample('YE').size().median()
rf /= DAYS

In [7]:
#with pd.ExcelWriter(OUTFILE) as writer:  


***

In [8]:
import yfinance as yf

/Users/markhendricks/Projects/mlp-portfolio-mgmt-2025/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
TICKS = [
    'USO',
    'UUP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [10]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [11]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
USO,United States Oil Fund,ETF,USD,2027874,1042997568,USO invests primarily in futures contracts for...
UUP,Invesco DB USD Index Bullish Fu,ETF,USD,1279773,426569600,The fund invests in futures contracts in an at...
SHV,iShares Short Treasury Bond ETF,ETF,USD,7305725,19008184320,The fund will invest at least 80% of its asset...


In [12]:
STARTDATE = '2015-01-01'
ENDDATE = '2024-12-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE, auto_adjust=False)['Adj Close']

[*********************100%***********************]  3 of 3 completed


In [13]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)
prices.index.name = 'date'

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_43968/4268833471.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


***

# Export Data

In [14]:
with pd.ExcelWriter(OUTFILE) as writer:  
    rf.to_excel(writer, sheet_name= 'risk-free rates', index=True)
    fx.to_excel(writer, sheet_name= 'exchange rates', index=True)
    #info.to_excel(writer, sheet_name= 'descriptions')
    retsx.to_excel(writer, sheet_name='excess returns')